## 1. Obtención de los datos

En esta sección, se carga los datos desde la base de datos y se va a convertir la serie temporal de ventas en un ejercicio Supervisado.

### 1.1 Cargar los datos desde la base de datos

Para la conexión con la base de datos se usa las librerías de `pandas` para mejorar los datos y `sqlalchemy` para la conexión con al base de datos.

In [62]:
##Para borrar
import pandas as pd
import sqlalchemy as sa


class DatabaseConnection:

    def __init__(
        self,
        username: str,
        password: str,
        database: str = "sandbox",
        host: str = "database-1.cjyy8w6ya3fr.eu-north-1.rds.amazonaws.com",
        port: int = 3306,
        dialect: str = "mysql",
        driver: str = "pymysql",
    ) -> None:
        """Creates a connection to a database

        Args:
            username (str): username
            password (str): password
            dialect (str, optional): dialect. Defaults to "mysql".
            driver (str, optional): driver. Defaults to "pymysql".
            host (str, optional): host. Defaults to "database-1.crek3tiqyj7r.eu-west-3.rds.amazonaws.com".
            port (int, optional): port. Defaults to 3306.
            database (str, optional): database. Defaults to "data".
        """
        connection_string = f"{dialect}+{driver}://{username}:{password}@{host}:{port}/{database}"
        self.engine = sa.create_engine(connection_string)

    def insert_dataframe(self, df: pd.DataFrame, table_name: str) -> None:
        """Inserts a dataframe into a table

        Args:
            df (pd.DataFrame): dataframe to insert
            table_name (str): table name
        """
        df.to_sql(table_name, self.engine, if_exists="replace", index=False)

    def query_to_df(self, query: str) -> pd.DataFrame:
        """Retrieves a dataframe from a query.

        Args:
            query (str): query to perform.

        Returns:
            pd.DataFrame: daframe with the results of the query.
        """
        with self.engine.connect() as conn:
            df = pd.read_sql_query(query, conn)
            return df

    def check_connection(self) -> bool:
        """Checks if the connection is working

        Returns:
            bool: True if the connection is working, False otherwise
        """
        try:
            self.engine.connect()
        except Exception as e:
            print(e)
            return False

In [63]:
##para borrar
DB_USERNAME = "usuario1"
DB_PASSWORD = "C0d35p4ce."
LOCATION = (36.721477644071705, -4.363132134392174)

In [3]:
import pandas as pd
import sqlalchemy as sa


class DatabaseConnection:

    def __init__(
        self,
        username: str,
        password: str,
        database: str = "##",
        host: str = "##",
        port: int = ##,
        dialect: str = "mysql",
        driver: str = "pymysql",
    ) -> None:
        """Creates a connection to a database

        Args:
            username (str): username
            password (str): password
            dialect (str, optional): dialect. Defaults to "mysql".
            driver (str, optional): driver. Defaults to "pymysql".
            host (str, optional): host. Defaults to "##".
            port (int, optional): port. Defaults to ##.
            database (str, optional): database. Defaults to "data".
        """
        connection_string = f"{dialect}+{driver}://{username}:{password}@{host}:{port}/{database}"
        self.engine = sa.create_engine(connection_string)

    def insert_dataframe(self, df: pd.DataFrame, table_name: str) -> None:
        """Inserts a dataframe into a table
        
        Args:
            df (pd.DataFrame): dataframe to insert
            table_name (str): table name
        """
        df.to_sql(table_name, self.engine, if_exists="replace", index=False)

    def query_to_df(self, query: str) -> pd.DataFrame:
        """Retrieves a dataframe from a query.

        Args:
            query (str): query to perform.

        Returns:
            pd.DataFrame: daframe with the results of the query.
        """
        with self.engine.connect() as conn:
            df = pd.read_sql_query(query, conn)
            return df

    def check_connection(self) -> bool:
        """Checks if the connection is working

        Returns:
            bool: True if the connection is working, False otherwise
        """
        try:
            self.engine.connect()
        except Exception as e:
            print(e)
            return False

SyntaxError: invalid syntax (2484462161.py, line 14)

In [ ]:
DB_USERNAME = "###"
DB_PASSWORD = "####"
LOCATION = (###, ###)

Conexión con la base de datos

In [64]:
database = DatabaseConnection(DB_USERNAME, DB_PASSWORD)
database.check_connection()

Se realiza una consulta SQL para obtener los datos y se comprueba que funciona correctamente con un .info

In [65]:
query = """
    SELECT familia,
        articulo,
        fechaVenta AS fecha_venta,
        festivo,
        precio,
        cantidad,
        importe,
        orden_articulo_familia

    FROM jpra_ventas_diarias_estudio
"""

df = database.query_to_df(query)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10322 entries, 0 to 10321
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   familia                 10322 non-null  object 
 1   articulo                10322 non-null  object 
 2   fecha_venta             10322 non-null  object 
 3   festivo                 809 non-null    object 
 4   precio                  10322 non-null  float64
 5   cantidad                10322 non-null  float64
 6   importe                 10322 non-null  float64
 7   orden_articulo_familia  10322 non-null  int64  
dtypes: float64(3), int64(1), object(4)
memory usage: 645.3+ KB


Se muestra los primeros valores.

In [66]:
df.head()

,familia,articulo,fecha_venta,festivo,precio,cantidad,importe,orden_articulo_familia
0,BOLLERIA,3880,2021-05-01,None,2.591,285.000,738.434998,3
1,BOLLERIA,3960,2021-05-01,None,2.318,402.000,931.835995,1
2,BOLLERIA,5803,2021-05-01,None,2.727,237.000,646.298981,5
3,BOLLERIA,6286,2021-05-01,None,3.136,165.000,517.440006,4
4,BOLLERIA,6425,2021-05-01,None,31.364,31.725,995.022083,2


Convertimos las fechas a formato datetime y los artículos a tipo int.

In [67]:
df["fecha_venta"] = pd.to_datetime(df["fecha_venta"])
df["articulo"] = df["articulo"].astype(int)

Se organiza los datos por fecha y se reinicia el índice.

In [68]:
df.sort_values(["fecha_venta"], inplace=True)
df.reset_index(inplace=True, drop=True)
df.head()

,familia,articulo,fecha_venta,festivo,precio,cantidad,importe,orden_articulo_familia
0,BOLLERIA,3880,2021-05-01,None,2.591,285.0,738.434998,3
1,PANADERIA,417,2021-05-01,None,4.038,72.0,290.736004,5
2,PANADERIA,1084,2021-05-01,None,0.865,423.0,365.895005,4
3,PANADERIA,1043,2021-05-01,None,2.596,327.0,848.892002,2
4,PASTELERIA,5403,2021-05-01,None,34.091,3.0,102.273003,2


Nota: Todo lo relacionado al EDA está en el notebook [EDA](./DataScience-Project/Datathon_Bakery/2.%20EDA.ipynb)

### 1.2 Convertir la serie en formato supervisado

Para predecir las ventas, se necesita transformar los datos a un problema supervisado, para ello:    
- Se deberá obtener la `cantidad` contabilizada de los días anteriores y añadirlos al día presente.
- Se deberá obtener `cantidad` contabilizada de las semanas anteriores y añadirlos al día presente.

Se diseña una función que permite transformar la serie a formato supervisado.
La función recibe como parámetros el dataframe, número de días y semanas a considerar. 

In [69]:
def multi_product_series_to_supervised(
    df: pd.DataFrame,
    target_column: str = "cantidad",
    id_column: str = "articulo",
    daily_lag: int = 1,
    weekly_lag: int = 1,
) -> pd.DataFrame:
    """
    Converts multiple time series (grouped by product) into a supervised learning dataset.
    
    Args:
        df: Input DataFrame with columns: fecha_venta, articulo, cantidad, etc.
        target_column: Name of the target variable column (default: "cantidad").
        id_column: Name of the column identifying products (default: "articulo").
        daily_lag: Number of daily lags to include (default: 1).
        weekly_lag: Number of weekly lags to include (default: 1).
    
    Returns:
        DataFrame with lagged features for each product.
    """
    # Verificar que la columna de fecha existe
    if 'fecha_venta' not in df.columns:
        raise ValueError("El DataFrame debe contener una columna 'fecha_venta'.")

    resultados = []
    
    # Ordenar por artículo y fecha
    df = df.sort_values([id_column, 'fecha_venta'])
    
    for articulo in df[id_column].unique():
        # Filtrar datos para el artículo actual
        df_articulo = df[df[id_column] == articulo].copy()
        
        # Crear características de lag para este artículo
        target_feature = df_articulo[target_column]
        
        # Función interna para crear lags
        def create_shifted_feature(lag: int, frequency: int, freq_label: str):
            shifted_feature = target_feature.shift(lag * frequency)
            feature_name = f"{target_column}_lag_{lag}_{freq_label}"
            df_articulo[feature_name] = shifted_feature
        
        # Añadir lags semanales (t-7, t-14, ...)
        for lag in range(1, weekly_lag + 1):
            create_shifted_feature(lag, 7, "weeks")
        
        # Añadir lags diarios (t-1, t-2, ...)
        for lag in range(1, daily_lag + 1):
            create_shifted_feature(lag, 1, "days")
        
        resultados.append(df_articulo)
    
    # Combinar todos los DataFrames y resetear índice
    return pd.concat(resultados).reset_index(drop=True)

In [70]:
df_supervised = multi_product_series_to_supervised(
    df=df,  # <- Aquí usas tu DataFrame real
    target_column="cantidad",
    id_column="articulo",
    daily_lag=6,  # Lags diarios: t-1, t-2, ..., t-6
    weekly_lag=4   # Lags semanales: t-7, t-14, t-21, t-28
)
df_supervised.head()

,familia,articulo,fecha_venta,festivo,precio,cantidad,importe,orden_articulo_familia,cantidad_lag_1_weeks,cantidad_lag_2_weeks,cantidad_lag_3_weeks,cantidad_lag_4_weeks,cantidad_lag_1_days,cantidad_lag_2_days,cantidad_lag_3_days,cantidad_lag_4_days,cantidad_lag_5_days,cantidad_lag_6_days
0,PANADERIA,417,2021-05-01,None,4.038,72.0,290.736004,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,PANADERIA,417,2021-05-02,Día de la Madre,4.038,81.0,327.077995,5,NaN,NaN,NaN,NaN,72.0,NaN,NaN,NaN,NaN,NaN
2,PANADERIA,417,2021-05-03,Día de la Cruz,4.038,90.0,363.419998,5,NaN,NaN,NaN,NaN,81.0,72.0,NaN,NaN,NaN,NaN
3,PANADERIA,417,2021-05-04,None,4.038,84.0,339.192001,5,NaN,NaN,NaN,NaN,90.0,81.0,72.0,NaN,NaN,NaN
4,PANADERIA,417,2021-05-05,None,4.038,84.0,339.192003,5,NaN,NaN,NaN,NaN,84.0,90.0,81.0,72.0,NaN,NaN


In [71]:
df_supervised.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10322 entries, 0 to 10321
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   familia                 10322 non-null  object        
 1   articulo                10322 non-null  int64         
 2   fecha_venta             10322 non-null  datetime64[ns]
 3   festivo                 809 non-null    object        
 4   precio                  10322 non-null  float64       
 5   cantidad                10322 non-null  float64       
 6   importe                 10322 non-null  float64       
 7   orden_articulo_familia  10322 non-null  int64         
 8   cantidad_lag_1_weeks    10217 non-null  float64       
 9   cantidad_lag_2_weeks    10112 non-null  float64       
 10  cantidad_lag_3_weeks    10007 non-null  float64       
 11  cantidad_lag_4_weeks    9902 non-null   float64       
 12  cantidad_lag_1_days     10307 non-null  float6

## 2. Procesamiento de los datos

En este paso se creará un Flujo de Transformación de Datos (Data Pipeline).

### 2.1 Extraer info de la fecha de venta

Con este campo se va a crear las siguientes columnas:

- dayofweek: Día de la semana.
- day: Día del mes.
- month: El mes.
- year: El año.
- weekend: Si es fin de semana.

Se va a crear la siguiente función para extraer los nuevos campos y adicionarlos al df.

In [72]:
def add_date_columns(df: pd.DataFrame, date_column: str = "fecha_venta") -> pd.DataFrame:
    df["day"] = df[date_column].dt.day
    df["month"] = df[date_column].dt.month
    df["year"] = df[date_column].dt.year
    df["dayofweek"] = df[date_column].dt.dayofweek + 1
    df["isWeekend"] = df["dayofweek"] >= 6
    return df

df_supervised_tmp = add_date_columns(df_supervised.copy())
df_supervised_tmp.head()

,familia,articulo,fecha_venta,festivo,precio,cantidad,importe,orden_articulo_familia,cantidad_lag_1_weeks,cantidad_lag_2_weeks,...,cantidad_lag_2_days,cantidad_lag_3_days,cantidad_lag_4_days,cantidad_lag_5_days,cantidad_lag_6_days,day,month,year,dayofweek,isWeekend
0,PANADERIA,417,2021-05-01,None,4.038,72.0,290.736004,5,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1,5,2021,6,True
1,PANADERIA,417,2021-05-02,Día de la Madre,4.038,81.0,327.077995,5,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2,5,2021,7,True
2,PANADERIA,417,2021-05-03,Día de la Cruz,4.038,90.0,363.419998,5,NaN,NaN,...,72.0,NaN,NaN,NaN,NaN,3,5,2021,1,False
3,PANADERIA,417,2021-05-04,None,4.038,84.0,339.192001,5,NaN,NaN,...,81.0,72.0,NaN,NaN,NaN,4,5,2021,2,False
4,PANADERIA,417,2021-05-05,None,4.038,84.0,339.192003,5,NaN,NaN,...,90.0,81.0,72.0,NaN,NaN,5,5,2021,3,False


### 2.2 Agregar columnas adicionales

Se agrega las siguientes columnas:

- `isHoliday`: Indica con un True y False si es un día festivo o no.
- `isBridgeDay`: Solo aplica para días que son puentes cuando el festivo es un jueves, el valor se marcará True un viernes y si el festivo es martes, el lunes se marcará como True.
- `preHoliday`: Indica True si el día es un día anterior a un festivo.
- `postHoliday`: Indica True si hay un festivo el día anterior.

Además la función elimina la columna de `festivo` porque ya se está usando la columna `isHoliday`


In [73]:
from datetime import timedelta
import pandas as pd

def add_holiday_features(
    df: pd.DataFrame,
    date_column: str = "fecha_venta",
    raw_holiday_column: str = "festivo",
    holiday_column: str = "isHoliday"
) -> pd.DataFrame:
    """
    Añade columna booleana de festivo (`isHoliday`) y contexto:
    - isBridgeDay: Viernes si el festivo es jueves, o lunes si es martes.
    - preHoliday: Día anterior a un festivo.
    - postHoliday: Día posterior a un festivo.
    """
    df = df.copy()

    # Convertir fechas si es necesario
    df[date_column] = pd.to_datetime(df[date_column], errors='coerce')

    # Crear columna isHoliday desde la columna original de festivos
    if raw_holiday_column in df.columns:
        df[holiday_column] = ~df[raw_holiday_column].isna()
        df.drop(columns=[raw_holiday_column], inplace=True)
    else:
        print(f"Advertencia: Columna '{raw_holiday_column}' no encontrada. No se marcarán festivos.")
        df[holiday_column] = False

    # Inicializar columnas de contexto
    df["isBridgeDay"] = False
    df["preHoliday"] = False
    df["postHoliday"] = False

    # Identificar fechas festivas
    holiday_dates = df.loc[df[holiday_column], date_column].dropna().unique()

    for holiday in holiday_dates:
        try:
            holiday = pd.to_datetime(holiday)

            pre_day = holiday - timedelta(days=1)
            post_day = holiday + timedelta(days=1)

            df.loc[df[date_column] == pre_day, "preHoliday"] = True
            df.loc[df[date_column] == post_day, "postHoliday"] = True

            if holiday.weekday() == 3:  # Jueves -> puente viernes
                df.loc[df[date_column] == (holiday + timedelta(days=1)), "isBridgeDay"] = True
            elif holiday.weekday() == 1:  # Martes -> puente lunes
                df.loc[df[date_column] == (holiday - timedelta(days=1)), "isBridgeDay"] = True

        except Exception as e:
            print(f"Error procesando fecha {holiday}: {str(e)}")

    return df


df_supervised_tmp = add_holiday_features(df_supervised.copy())
df_supervised_tmp.head()

,familia,articulo,fecha_venta,precio,cantidad,importe,orden_articulo_familia,cantidad_lag_1_weeks,cantidad_lag_2_weeks,cantidad_lag_3_weeks,...,cantidad_lag_1_days,cantidad_lag_2_days,cantidad_lag_3_days,cantidad_lag_4_days,cantidad_lag_5_days,cantidad_lag_6_days,isHoliday,isBridgeDay,preHoliday,postHoliday
0,PANADERIA,417,2021-05-01,4.038,72.0,290.736004,5,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,False,False,True,False
1,PANADERIA,417,2021-05-02,4.038,81.0,327.077995,5,NaN,NaN,NaN,...,72.0,NaN,NaN,NaN,NaN,NaN,True,False,True,False
2,PANADERIA,417,2021-05-03,4.038,90.0,363.419998,5,NaN,NaN,NaN,...,81.0,72.0,NaN,NaN,NaN,NaN,True,False,False,True
3,PANADERIA,417,2021-05-04,4.038,84.0,339.192001,5,NaN,NaN,NaN,...,90.0,81.0,72.0,NaN,NaN,NaN,False,False,False,True
4,PANADERIA,417,2021-05-05,4.038,84.0,339.192003,5,NaN,NaN,NaN,...,84.0,90.0,81.0,72.0,NaN,NaN,False,False,False,False


### 2.3 Se agrega Información meteorológica


Se utiliza la API de meteostat para obtener los datos meteorológicos.
Se añadirá la columna `isRaining` para indicar si ese día hubo lluvia o no. Los valores serán verdader o falso.

In [74]:
from datetime import datetime
from meteostat import Point, Daily
import numpy as np


def get_weather_df(start: datetime, end: datetime, location: tuple[int, int]) -> pd.DataFrame:
    lat, lon = location
    point = Point(lat, lon)
    return Daily(point, start, end).fetch().fillna(0).reset_index()

def merge_dfs_by_column(df1: pd.DataFrame, df2: pd.DataFrame, col_1_name: str, col_2_name: str) -> pd.DataFrame:
    merged_df = df1.reset_index().merge(df2, left_on=col_1_name, right_on=col_2_name, how="inner").set_index("index")
    merged_df.index.name = None
    return merged_df

def add_rain_column(df: pd.DataFrame, precipitation_column_name: str = "prcp") -> pd.DataFrame:
    rain_condition = df[precipitation_column_name] > 0
    df["isRaining"] = np.where(rain_condition, True, False)
    return df

def add_weather_columns(
    df: pd.DataFrame,
    location: tuple[int, int],
    date_column_name: str = "fecha_venta",
) -> pd.DataFrame:
    start = df[date_column_name].min()
    end = df[date_column_name].max()
    weather_df = get_weather_df(start, end, location)
    weather_df = add_rain_column(weather_df)
    weather_df = merge_dfs_by_column(df, weather_df, date_column_name, "time")
    return weather_df


df_supervised_tmp = df_supervised.copy()
add_weather_columns(df_supervised, LOCATION).head()


,familia,articulo,fecha_venta,festivo,precio,cantidad,importe,orden_articulo_familia,cantidad_lag_1_weeks,cantidad_lag_2_weeks,...,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun,isRaining
0,PANADERIA,417,2021-05-01,None,4.038,72.0,290.736004,5,NaN,NaN,...,11.9,22.8,0.0,0.0,0.0,14.2,0.0,1014.0,0.0,False
1,PANADERIA,417,2021-05-02,Día de la Madre,4.038,81.0,327.077995,5,NaN,NaN,...,10.8,23.2,0.0,0.0,0.0,15.7,0.0,1015.9,0.0,False
2,PANADERIA,417,2021-05-03,Día de la Cruz,4.038,90.0,363.419998,5,NaN,NaN,...,12.7,20.1,6.5,0.0,0.0,9.7,0.0,1017.7,0.0,True
3,PANADERIA,417,2021-05-04,None,4.038,84.0,339.192001,5,NaN,NaN,...,12.0,20.9,0.0,0.0,0.0,12.1,0.0,1018.0,0.0,False
4,PANADERIA,417,2021-05-05,None,4.038,84.0,339.192003,5,NaN,NaN,...,12.3,21.7,0.0,0.0,0.0,13.8,0.0,1016.5,0.0,False


In [91]:
##
# Crear una copia independiente del DataFrame original
df_final = df_supervised.copy()

# Aplicar las funciones en secuencia
df_final = add_date_columns(df_final)
df_final = add_holiday_features(df_final)
df_final = add_weather_columns(df_final, location=LOCATION) 

df_final.head()

,familia,articulo,fecha_venta,precio,cantidad,importe,orden_articulo_familia,cantidad_lag_1_weeks,cantidad_lag_2_weeks,cantidad_lag_3_weeks,...,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun,isRaining
0,PANADERIA,417,2021-05-01,4.038,72.0,290.736004,5,NaN,NaN,NaN,...,11.9,22.8,0.0,0.0,0.0,14.2,0.0,1014.0,0.0,False
1,PANADERIA,417,2021-05-02,4.038,81.0,327.077995,5,NaN,NaN,NaN,...,10.8,23.2,0.0,0.0,0.0,15.7,0.0,1015.9,0.0,False
2,PANADERIA,417,2021-05-03,4.038,90.0,363.419998,5,NaN,NaN,NaN,...,12.7,20.1,6.5,0.0,0.0,9.7,0.0,1017.7,0.0,True
3,PANADERIA,417,2021-05-04,4.038,84.0,339.192001,5,NaN,NaN,NaN,...,12.0,20.9,0.0,0.0,0.0,12.1,0.0,1018.0,0.0,False
4,PANADERIA,417,2021-05-05,4.038,84.0,339.192003,5,NaN,NaN,NaN,...,12.3,21.7,0.0,0.0,0.0,13.8,0.0,1016.5,0.0,False


## 3. Creación de Pipeline

### 3.1 Preprocesamiento de los datos de entrada

In [75]:
from sklearn.base import BaseEstimator, TransformerMixin, OneToOneFeatureMixin


class PreprocessTransformer(BaseEstimator, TransformerMixin, OneToOneFeatureMixin):
    def __init__(self, location: tuple[int, int], daily_lag: int, weekly_lag: int):
        self.location = location
        self.daily_lag = daily_lag
        self.weekly_lag = weekly_lag

    def fit(self, X: pd.DataFrame, y: pd.Series | None = None):
        return self

    def transform(self, X: pd.DataFrame) -> pd.DataFrame:
        X = X.copy()  # avoid modifying incoming data
        X = add_date_columns(X)
        X = add_holiday_features(X)
        X = add_weather_columns(X, location=self.location)
        return X

In [83]:
##
from sklearn.base import TransformerMixin, BaseEstimator, OneToOneFeatureMixin

class PreprocessTransformer(OneToOneFeatureMixin, BaseEstimator, TransformerMixin):
    def __init__(self, location: tuple[int, int], daily_lag: int, weekly_lag: int):
        self.location = location
        self.daily_lag = daily_lag
        self.weekly_lag = weekly_lag

    def fit(self, X: pd.DataFrame, y: pd.Series | None = None):
        return self

    def transform(self, X: pd.DataFrame) -> pd.DataFrame:
        X = X.copy()
        # Añadir features
        X = add_date_columns(X)
        X = add_holiday_features(X)
        X = add_weather_columns(X, location=self.location)
        
        # --- Limpieza dentro del transformador ---
        # Eliminar columnas problemáticas
        cols_to_drop = [
            'fecha_venta', 'time', 'articulo', 'familia',  # No numéricas
            'snow', 'wdir', 'wpgt', 'pres', 'tsun',  # Meteorológicas irrelevantes
            'festivo'  # Redundante con isHoliday
        ]
        return X.drop(columns=cols_to_drop, errors='ignore')

In [84]:
preprocessor = PreprocessTransformer(location=LOCATION, daily_lag=6, weekly_lag=4)
preprocessed_df = preprocessor.transform(df_supervised)
preprocessed_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10322 entries, 0 to 10321
Data columns (total 29 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   precio                  10322 non-null  float64
 1   cantidad                10322 non-null  float64
 2   importe                 10322 non-null  float64
 3   orden_articulo_familia  10322 non-null  int64  
 4   cantidad_lag_1_weeks    10217 non-null  float64
 5   cantidad_lag_2_weeks    10112 non-null  float64
 6   cantidad_lag_3_weeks    10007 non-null  float64
 7   cantidad_lag_4_weeks    9902 non-null   float64
 8   cantidad_lag_1_days     10307 non-null  float64
 9   cantidad_lag_2_days     10292 non-null  float64
 10  cantidad_lag_3_days     10277 non-null  float64
 11  cantidad_lag_4_days     10262 non-null  float64
 12  cantidad_lag_5_days     10247 non-null  float64
 13  cantidad_lag_6_days     10232 non-null  float64
 14  day                     10322 non-null  int

### 3.2 Normalización de los datos de entrada

Antes de hacer indicar los valores, se eliminaran las siguientes columnas porque no aportan ningún valor en nuestro dataframe, ya que éstas columnas tienen valores 0 `snow`, `wdir` y `tsun`.

Y la columna `prcp` porque se utilizó para crear la columna `isRaining`

In [77]:
preprocessed_df = preprocessed_df.drop(columns=["snow", "wdir", "tsun", "prcp"])
preprocessed_df.columns

Index(['familia', 'articulo', 'fecha_venta', 'precio', 'cantidad', 'importe',
       'orden_articulo_familia', 'cantidad_lag_1_weeks',
       'cantidad_lag_2_weeks', 'cantidad_lag_3_weeks', 'cantidad_lag_4_weeks',
       'cantidad_lag_1_days', 'cantidad_lag_2_days', 'cantidad_lag_3_days',
       'cantidad_lag_4_days', 'cantidad_lag_5_days', 'cantidad_lag_6_days',
       'day', 'month', 'year', 'dayofweek', 'isWeekend', 'isHoliday',
       'isBridgeDay', 'preHoliday', 'postHoliday', 'time', 'tavg', 'tmin',
       'tmax', 'wspd', 'wpgt', 'pres', 'isRaining'],
      dtype='object')

Para la normalización de los datos se utiliza `OneHotEncoder` para las variables categoricas y `MinMaxScaler`. para las variables numéricas.

In [78]:
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer, make_column_selector


categorical_features = make_column_selector(dtype_include=[object, np.int32])
numerical_features = make_column_selector(dtype_include=[np.float64])

ohe_encoder = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
minmax_scaler = MinMaxScaler(feature_range=(0, 1))

encoders_transformer = ColumnTransformer(
    transformers=[
        ("OneHotEncoder", ohe_encoder, categorical_features),
        ("MinMaxScaler", minmax_scaler, numerical_features),
    ],
)

In [85]:
##

from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer, make_column_selector
import numpy as np

# Selectores modificados según tus necesidades reales
categorical_features = make_column_selector(dtype_include=['bool'])  # Solo columnas booleanas
numerical_features = make_column_selector(dtype_include=np.number)   # Todos los números (int y float)

# Mismos encoders que ya tenías
ohe_encoder = OneHotEncoder(sparse_output=False, handle_unknown="ignore", drop='if_binary')  # Optimizado para booleanas
minmax_scaler = MinMaxScaler(feature_range=(0, 1))

# Transformer mejorado
encoders_transformer = ColumnTransformer(
    transformers=[
        ("OneHotEncoder", ohe_encoder, categorical_features),
        ("MinMaxScaler", minmax_scaler, numerical_features),
    ],
    remainder='drop'  # Descarta lo no procesado explícitamente
)

### 3.3 Eliminar los valores nulos

En este paso se rellena los valores nulos calculando la media de cada columna.

In [79]:
from sklearn.impute import SimpleImputer


nans_remover = SimpleImputer(strategy="mean")
remove_nans_transformer = ColumnTransformer(
    transformers=[
        ("RemoveNans", nans_remover, numerical_features),
    ],
)


In [86]:
##

from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import numpy as np

# Versión optimizada (misma estructura, solo cambio esencial)
nans_remover = SimpleImputer(strategy="median")  # Cambio a median para robustez
remove_nans_transformer = ColumnTransformer(
    transformers=[
        ("RemoveNans", nans_remover, make_column_selector(dtype_include=np.number)),  # Incluye todos los numéricos
    ],
    remainder="passthrough"  # Conserva columnas no numéricas
)

### 3.4 Incluir el modelo de Machine Learning

Como nos encontramos en un problema de regresión porque se quiere predecir las cantidades vendidas, se va a utilizar un modelo de regresión. En este caso, vamos a utilizar el modelo `LinearRegression` de Scikit-Learn.

In [87]:
from sklearn.linear_model import LinearRegression


model = LinearRegression()

### 3.5 Creamos el pipeline para las variables de entrada

Se agrega los pasos anteriores al pipeline de transformación de datos de forma secuencial. El pipeline se encargará de aplicar las transformaciones a los datos de entrada y de entrenar el modelo.

In [88]:
from sklearn.pipeline import Pipeline


input_pipeline = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        ("nan_remover", remove_nans_transformer),
        ("data_encoding", encoders_transformer),
        ("model", model),
    ]
)

input_pipeline.set_output(transform="pandas")

,steps,"[('preprocessor', ...), ('nan_remover', ...), ...]"
,transform_input,None
,memory,None
,verbose,False
,location,"(36.721477644071705, ...)"
,daily_lag,6
,weekly_lag,4
,transformers,"[('RemoveNans', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None


### 3.6 Normalizar la variable objetivo

En ultimo paso, se usa el mismo `OneHotEncoder` utilizado para los datos de entrada para codificar la variable objetivo (target), pero sin ajustarlo nuevamente. Esto asegura que la variable objetivo esté en el mismo formato que los datos de entrada, facilitando así la predicción y evaluación del modelo.

In [90]:
from sklearn.compose import TransformedTargetRegressor


target_feature_scaler = MinMaxScaler(feature_range=(0,1))
final_pipeline = TransformedTargetRegressor(
    regressor=input_pipeline,
    transformer=target_feature_scaler,
)
final_pipeline

,regressor,Pipeline(step...egression())])
,transformer,MinMaxScaler()
,func,None
,inverse_func,None
,check_inverse,True
,location,"(36.721477644071705, ...)"
,daily_lag,6
,weekly_lag,4
,transformers,"[('RemoveNans', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3


## 4. Entrenar el modelo

Se utiliza el pipeline creado anteriormente para entrenar el modelo. Marcamos el horizonte de tiempo del conjunto de test a 30 días. Esto significa que el modelo se evaluará con los datos de los últimos 30 días y se entrenará con los datos previos a dicho horizonte.

Se entrena el modelo por artículos porque no todos los artículos dentro de las familias se comportan de la misma manera.

In [251]:
def split_by_article(df: pd.DataFrame, article_col: str = "articulo") -> dict[int, pd.DataFrame]:
    """
    Divide el DataFrame en diccionario de DataFrames, uno por artículo.
    
    Args:
        df: DataFrame con todos los datos.
        article_col: Nombre de la columna que contiene los IDs de artículos.
    
    Returns:
        Un diccionario donde las claves son los IDs de artículos y los valores son DataFrames filtrados.
    """
    articles = df[article_col].unique()
    return {article: df[df[article_col] == article].copy() for article in articles}

In [252]:
dict_articulos = split_by_article(preprocessed_df, article_col="articulo")

In [253]:
print("Artículos disponibles:", dict_articulos.keys())

Artículos disponibles: dict_keys([np.int64(417), np.int64(900), np.int64(968), np.int64(1043), np.int64(1084), np.int64(3880), np.int64(3960), np.int64(5403), np.int64(5404), np.int64(5803), np.int64(6286), np.int64(6425), np.int64(6451), np.int64(6523), np.int64(6549)])


Se asignará los df creados en valores para poder entrenarlos por artículo

In [254]:
df_articulo_417 = dict_articulos[417]
df_articulo_900 = dict_articulos[900]
df_articulo_968 = dict_articulos[968]
df_articulo_1043 = dict_articulos[1043]
df_articulo_1084 = dict_articulos[1084]
df_articulo_3880 = dict_articulos[3880]
df_articulo_3960 = dict_articulos[3960]
df_articulo_5403 = dict_articulos[5403]
df_articulo_5404 = dict_articulos[5404]
df_articulo_5803 = dict_articulos[5803]
df_articulo_6286 = dict_articulos[6286]
df_articulo_6425 = dict_articulos[6425]
df_articulo_6451 = dict_articulos[6451]
df_articulo_6523 = dict_articulos[6523]
df_articulo_6549 = dict_articulos[6549]


In [255]:
df_articulo_1043.head()

,familia,articulo,fecha_venta,precio,cantidad,importe,orden_articulo_familia,cantidad_lag_1_weeks,cantidad_lag_2_weeks,cantidad_lag_3_weeks,...,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun,isRaining
2190,PANADERIA,1043,2021-05-01,2.596,327.0,848.892002,2,NaN,NaN,NaN,...,11.9,22.8,0.0,0.0,0.0,14.2,0.0,1014.0,0.0,False
2191,PANADERIA,1043,2021-05-02,2.596,303.0,786.587997,2,NaN,NaN,NaN,...,10.8,23.2,0.0,0.0,0.0,15.7,0.0,1015.9,0.0,False
2192,PANADERIA,1043,2021-05-03,2.596,216.0,560.735996,2,NaN,NaN,NaN,...,12.7,20.1,6.5,0.0,0.0,9.7,0.0,1017.7,0.0,True
2193,PANADERIA,1043,2021-05-04,2.596,177.0,459.491999,2,NaN,NaN,NaN,...,12.0,20.9,0.0,0.0,0.0,12.1,0.0,1018.0,0.0,False
2194,PANADERIA,1043,2021-05-05,2.596,180.0,467.279993,2,NaN,NaN,NaN,...,12.3,21.7,0.0,0.0,0.0,13.8,0.0,1016.5,0.0,False


Se guarda la familia a la que pertennece el producto para guaradrlo como tag en MLFlow

### 4.1 Pequeña limpieza de los datos

Se elimina columnas que no se van a utilizar en el modelo

In [256]:
drop_cols = ['importe', 'articulo', 'familia', 'orden_articulo_familia']

df_articulo_417 = df_articulo_417.drop(columns=drop_cols)
df_articulo_900 = df_articulo_900.drop(columns=drop_cols)
df_articulo_968 = df_articulo_968.drop(columns=drop_cols)
df_articulo_1043 =df_articulo_1043.drop(columns=drop_cols)
df_articulo_1084 =df_articulo_1084.drop(columns=drop_cols)
df_articulo_3880 =df_articulo_3880.drop(columns=drop_cols)
df_articulo_3960 =df_articulo_3960.drop(columns=drop_cols)
df_articulo_5403 =df_articulo_5403.drop(columns=drop_cols)
df_articulo_5404 =df_articulo_5404.drop(columns=drop_cols)
df_articulo_5803 =df_articulo_5803.drop(columns=drop_cols)
df_articulo_6286 =df_articulo_6286.drop(columns=drop_cols)
df_articulo_6425 =df_articulo_6425.drop(columns=drop_cols)
df_articulo_6451 =df_articulo_6451.drop(columns=drop_cols)
df_articulo_6523 =df_articulo_6523.drop(columns=drop_cols)
df_articulo_6549 =df_articulo_6549.drop(columns=drop_cols)

dfs_articulos = {
    417: df_articulo_417,
    900: df_articulo_900,
    968: df_articulo_968,
    1043: df_articulo_1043,
    1084: df_articulo_1084,
    3880: df_articulo_3880,
    3960: df_articulo_3960,
    5403: df_articulo_5403,
    5404: df_articulo_5404,
    5803: df_articulo_5803,
    6286: df_articulo_6286,
    6425: df_articulo_6425,
    6451: df_articulo_6451,
    6523: df_articulo_6523,
    6549: df_articulo_6549
}

In [257]:
dfs_articulos[417].columns

Index(['fecha_venta', 'precio', 'cantidad', 'cantidad_lag_1_weeks',
       'cantidad_lag_2_weeks', 'cantidad_lag_3_weeks', 'cantidad_lag_4_weeks',
       'cantidad_lag_1_days', 'cantidad_lag_2_days', 'cantidad_lag_3_days',
       'cantidad_lag_4_days', 'cantidad_lag_5_days', 'cantidad_lag_6_days',
       'day', 'month', 'year', 'dayofweek', 'isWeekend', 'isHoliday',
       'isBridgeDay', 'preHoliday', 'postHoliday', 'time', 'tavg', 'tmin',
       'tmax', 'prcp', 'snow', 'wdir', 'wspd', 'wpgt', 'pres', 'tsun',
       'isRaining'],
      dtype='object')

### 4.2. Se divide en conjuntos de entrenamiento y test.

In [258]:
from datetime import datetime

# Ya no es necesario eliminar las columnas aquí
# drop_cols = ['importe', 'articulo', 'familia', 'orden_articulo_familia']
# df_articulo_417 = df_articulo_417.drop(columns=drop_cols)
# ... y el resto de los DataFrames

# DICCIONARIO CON LOS DATAFRAMES
dfs_articulos 

# --- CÓDIGO MODIFICADO ---
# La función de preparación de datos ahora se encarga solo de la partición de los datos
def preparar_datos_para_modelado(
    dfs_articulos: dict,
    target_col: str = "cantidad",
    horizont: datetime = datetime(2023, 3, 31)
) -> dict:
    resultados = {}
    for id_articulo, df in dfs_articulos.items():
        try:
            mask_train = df["fecha_venta"] < horizont
            mask_test = df["fecha_venta"] >= horizont

            df_train = df.loc[mask_train].copy()
            df_test = df.loc[mask_test].copy()
            
            # Aquí es donde realmente usas 'fecha_venta' para el split, después ya no se necesita.
            X_train = df_train.drop(columns=[target_col])
            y_train = df_train[target_col]
            X_test = df_test.drop(columns=[target_col])
            y_test = df_test[target_col]

            resultados[id_articulo] = {
                "X_train": X_train,
                "y_train": y_train,
                "X_test": X_test,
                "y_test": y_test
            }
        except Exception as e:
            print(f"❌ Error procesando artículo {id_articulo}: {str(e)}")

    return resultados

# Ejecuta la preparación de datos con la función modificada
datos_preparados = preparar_datos_para_modelado(dfs_articulos)




Se define la familia para poder guardarlo como un tag en MLFlow

In [259]:
familias_por_articulo = {
    3960: "BOLLERIA",
    5803: "BOLLERIA",
    3880: "BOLLERIA",
    6286: "BOLLERIA",
    6425: "BOLLERIA",
    1084: "PANADERIA",
    417: "PANADERIA", 
    900: "PANADERIA",
    968: "PANADERIA",
    1043: "PANADERIA",
    6451: "PASTELERIAí",
    6549: "PASTELERIA",
    6523: "PASTELERIA",
    5403: "PASTELERIA",
    5404: "PASTELERIA"
}

### 4.3 Iniciar el servidor de MLFlow

In [260]:
import mlflow

# Connect to MLFlow server hosted locally
MLFLOW_EXPERIMENT_NAME = "Datathon V"
MLFLOW_TRACKING_URI = "http://localhost:5000"


mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment(MLFLOW_EXPERIMENT_NAME)

<Experiment: artifact_location='mlflow-artifacts:/523618784756416889', creation_time=1754591108677, experiment_id='523618784756416889', last_update_time=1754591108677, lifecycle_stage='active', name='Datathon V', tags={}>

In [261]:
print("✅ Pipeline configurado correctamente:")
print(final_pipeline)

✅ Pipeline configurado correctamente:
TransformedTargetRegressor(regressor=Pipeline(steps=[('preprocessor',
                                                      PreprocessTransformer(daily_lag=6,
                                                                            location=(36.721477644071705,
                                                                                      -4.363132134392174),
                                                                            weekly_lag=4)),
                                                     ('column_dropper',
                                                      ColumnDropperTransformer(columns_to_drop=['importe',
                                                                                                'articulo',
                                                                                                'familia',
                                                                                                'orden_a

In [262]:
import mlflow
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Tu diccionario de familias
familias_por_articulo = { ... }

# Configuración de MLflow
MLFLOW_EXPERIMENT_NAME = "Datathon V"
MLFLOW_TRACKING_URI = "http://localhost:5000"
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment(MLFLOW_EXPERIMENT_NAME)

# --- CÓDIGO MODIFICADO ---
def entrenar_modelo_final(id_articulo, datos_articulo):
    try:
        with mlflow.start_run(run_name=f"Artículo_{id_articulo}"):
            X_train = datos_articulo["X_train"]
            y_train = datos_articulo["y_train"]
            X_test = datos_articulo["X_test"]
            y_test = datos_articulo["y_test"]

            # Ya no es necesario el filtro de columnas, el pipeline lo maneja
            # X_train = X_train.select_dtypes(...)
            # X_test = X_test.select_dtypes(...)
            
            if len(X_train) == 0 or len(X_test) == 0:
                print(f"⚠️ Artículo {id_articulo}: Sin datos suficientes")
                return None
            
            familia = familias_por_articulo.get(id_articulo, "Desconocida")
            mlflow.set_tags({
                "articulo": str(id_articulo),
                "familia": familia,
                "modelo": "LinearRegression+PipelineCompleto"
            })
            
            final_pipeline.fit(X_train, y_train)
            
            y_pred = final_pipeline.predict(X_test)
            
            metrics = {
                "mae_test": mean_absolute_error(y_test, y_pred),
                "rmse_test": np.sqrt(mean_squared_error(y_test, y_pred)),
                "r2_test": r2_score(y_test, y_pred)
            }
            
            mlflow.log_metrics(metrics)
            mlflow.sklearn.log_model(final_pipeline, "model")
            
            print(f"✅ {id_articulo} ({familia}) - MAE: {metrics['mae_test']:.2f}")
            return metrics
            
    except Exception as e:
        print(f"❌ Error en artículo {id_articulo}: {str(e)}")
        return None

# --- Bucle para entrenar los modelos ---
resultados_finales = {}
for id_articulo, datos_articulo in datos_preparados.items():
    print(f"\n🔍 Procesando artículo {id_articulo}...")
    resultado = entrenar_modelo_final(id_articulo, datos_articulo)
    if resultado is not None:
        resultados_finales[id_articulo] = resultado

# Mostrar resumen
print("\n🎯 Resultados Finales:")
for art, res in resultados_finales.items():
    print(f"Artículo {art}: MAE={res['mae_test']:.2f} | R²={res['r2_test']:.2f}")



🔍 Procesando artículo 417...
🏃 View run Artículo_417 at: http://localhost:5000/#/experiments/523618784756416889/runs/30cab881750045fea2aa9df2dc96c094
🧪 View experiment at: http://localhost:5000/#/experiments/523618784756416889
❌ Error en artículo 417: 'set' object has no attribute 'get'

🔍 Procesando artículo 900...
🏃 View run Artículo_900 at: http://localhost:5000/#/experiments/523618784756416889/runs/71dfbdd617514827bd615e005285f466
🧪 View experiment at: http://localhost:5000/#/experiments/523618784756416889
❌ Error en artículo 900: 'set' object has no attribute 'get'

🔍 Procesando artículo 968...
🏃 View run Artículo_968 at: http://localhost:5000/#/experiments/523618784756416889/runs/c9dc72c19a1a44a9b6eeafac94f79ef9
🧪 View experiment at: http://localhost:5000/#/experiments/523618784756416889
❌ Error en artículo 968: 'set' object has no attribute 'get'

🔍 Procesando artículo 1043...
🏃 View run Artículo_1043 at: http://localhost:5000/#/experiments/523618784756416889/runs/c876fcf300774